# Brief intro to deep nueral networks

In [ ]:
from IPython.display import clear_output

clear_output()

<img src=https://github.com/bharris12/URP_2021_Programming_Course/raw/main/lecture_8/figure/table.png>

for this tutorial we will focus on supervised learning. More specifically, we will use some classification models to expain the main concepts of deep learning

A deep learning model is constructed by three main parts,model structure, loss function, and optimization.
<img src=https://eenews.cdnartwhere.eu/sites/default/files/images/01-picture-library/ChristophHammerschmidt/2016-11-november/Cadence_Hijazi/cnn_fig1.jpeg />

The weight is adjusted by a process called gradient descent

<img src=https://miro.medium.com/max/1400/1*IfVAzJJWIrCw0saOZAxpzA.png />

## Mutilayer Perceptron (MLP)

The most basic model structure is what we call Multilayer Perceptron
<img src=https://miro.medium.com/max/600/1*xxZXeKfVKTRqh54t10815A.jpeg />
Its constructed by basic units that are called 'Neurons' or 'Perceptrons'. Each individual one is actually very simply constructed.
<img src=https://pythonmachinelearning.pro/wp-content/uploads/2017/09/Single-Perceptron.png.webp />

Although each one is a very simple function approximater, by layering them together, the model now has a much greater power and flexibility in approximating functions. But after all, neural networks can be thing of as a function approximator, but with less assumptions.
A fun app for visualiztion MLP: https://playground.tensorflow.org/

## Convolutional Layer

MLP isn't the best fit on all problems. Especially things like images. How do we get 'feature' from images? Per pixel? Thats just way too much. So another kind of layer comes into play.
![ChessUrl](https://miro.medium.com/max/1052/0*jLoqqFsO-52KHTn9.gif)

We call the weight we have learnt 'filters'. It to some extent represents what pattern the model has learnt to look for. And there are usually multiple filters per convolution layer, give the model the ability to learn more than one pattern.

<img src=https://www.researchgate.net/publication/336805909/figure/fig1/AS:817888827023360@1572011300751/Schematic-diagram-of-a-basic-convolutional-neural-network-CNN-architecture-26.ppm />

## Pooling layer

As seen in the above figure structure, pooling layers often exist in deep learning models. Its main usage is to condense information. There are several ways pooling can be done:

<img src=https://miro.medium.com/max/1400/1*oP-lySI7atBfDgpWQjnB6w.png />

# Neural Network implementation

TensorFlow is a widely used deep learning framework. Meaning that all the function/layers that we have mentioned above is already implemented as easy-to-use functions. For this class we will especially depend on TensorFlow.keras.

There are lots of classes/blogs online for tensorflow tutorial. Tensorflow itself also come with a few tutorials:
https://www.tensorflow.org/tutorials

The model that we are going to go through will use real biology data

# Convolutional Neural Networks for regulatory genomics

In this notebook, we will walk through how to build, train, and evaluate a convolutional neural network (CNN) for the computational task of predicting transcription factor binding sites from Chromatin Immunoprecipitation sequencing (ChIP-seq) data.



In [ ]:
import h5py # python module needed to load dataset (stored in an hdf5 file format)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


#### Overview
Here, we will employ a CNN model to experimental ChIP-seq data for CTCF, a transciption factor that is known to bind to DNA and help form loops. We will set this up as a supervised learning task, specifically a binary classification, where we give the model a sequence and it has to predict whether or not it is associated with TF binding (i.e. presence or absence of a ChIP-seq peak). Thus, we need a set of background sequences which don't have a ChIP-seq peak. 

________________________________________________________________________________
#### How should we choose background sequences?

Randomly choosing sequences throughout the genome may include a CTCF motif (the accessibility of DNA is different across cell-types). If the properties of the background sequences are different from the positive sequences (e.g. GC-content differences), then a model may learn this instead of the motif patterns. We could just shuffle the sequences, but we may end up with a model that learns real genomic sequences vs fake genomic sequences. One background sequence that makes sense is to use accessible chromatin in the same cell type BUT does not overlap with the TF binding sites from ChIP-seq.  This way, the background sequences come from regulatory DNA from the same cell type and the only presumable difference between background and positive sequences is the TF binding site.  

We already did this and processed the dataset:

Let's download the data:


In [ ]:
!wget https://www.dropbox.com/s/0ks4dk6md2l9mqx/CTCF_200.h5

For this dataset, the keys for the train sequences is x_train and its corresponding label is y_train.  Note that the transpose changes the shape of the sequences such that the shape is (N,L,A), where N is the number of sequences, L is the length of the sequence=200, and A is the alphabet size=4). 

We define the alphabet which was used to generate the dataset (dim0=A, dim1=C, dim2=G, dim3=T)



In [ ]:
# load dataset
file_path = 'CTCF_200.h5'

# print shape of training data


There should be 51,156 training sequences each 200 nts long with an alphabet size of 4. Let's see what a sequence actually looks like:

Now let's look at the shape of the training labels (y).

# Convolutional Neural Network Overview

A convolutional neural network (CNN) is composed of a stack of convolutional modules, each of which perform feature extraction from the previous layer. Each module consists of a convolutional layer, non-linear activation, followed by an optional pooling layer. The convolutional modules are usually followed by one or more dense layers which help it to learn more complex features. The final dense layer in a CNN is where predictions of the model are made. The activations of the output layer depend on the task at hand. In binary classification of TF binding sites, it will be a sigmoid activation (similar to logistic regression).

### Convolutional layer

Convolutional layers employ a specified number of convolution filters. The output of the convolutional filter scans is called a feature map. Convolutional layers then typically apply a ReLU activation function to the feature maps. Convolutional layers have 4 main hyperparameter choices: number of filters, filter size (i.e. kernel_size), padding, and activation. 

Eg. Conv1D(filters=24, kernel_size=19, activation='relu', padding='same')

- The number of filters specifies how many patterns you aim to capture in that layer.  
- The kernel_size sets how large you expect the important patterns to be in the given layer. 
- The padding argument specifies one of two enumerated values: valid (default) or same. To specify that the output tensor should have the same width and height values as the input tensor, we set padding=same, which instructs TensorFlow to pad the sequences with 0 values at the edges to preserve the shape of the sequence. 
- The activation sets the non-linearity added after the convolutional scans. The default is linear. We will use rectified linear units (ReLU) here. Other choices include: sigmoid, tanh, elu, softplus, swish, prelu, leaky relu.

### Max-pool layer

Max-pooling is a technique to downsample the data over a window size set by pool_size. This allows for deeper convolutional layers to learn patterns over a larger sequence context, while significantly reducing the processing time. The strides of max pooling sets how many steps to take for each max-pool operation. If the strides is set to the pool_size, this creates non-overlapping windows of the feature maps. Here, we will downsample the 1st convolutional layer scans by non-overlapping windows of size 25. This will result in a shape of (N, L/25, A).

Eg. layers.MaxPool1D(pool_size=25)

### fully-connected hidden layer

Dense (fully connected) layers, connect every node in the layer (set by num_units) to every node in the preceding layer. The hyperparameters of interest are the number of neurons (units) and the activation, which we will set to relu.

layers.Dense(units=64, activation='relu')

FYI, most hyperparameter settings for convolutional, max-pooling, and dense layers are chosen through trail-and-error. Most settings result in similar performance -- they are all nearly just as good as each other. Finding the jackpot setting usually means you improve accuracy from 0.9 to 0.92.  


### fully-connected output layer

The output layer is a linear classifier model. So, the number of predictions we want to make is the number of output neurons we need to create. Here that number is one, because we are only making a single prediction -- whether or not a sequence is associated with a CTCF ChIP-seq peak. If there were more labels, all we have to do is set the 1 to an appropriate value. Note that the activation is dependent on what you are trying to predict. For instance, a regression can be achieved by a simple linear model, without any activation; a binary classificaiton can be achieved with a sigmoid activation, a multi-class prediction can be achieved with a softmax activation. Here we are performing a single-class binary classification, so we will employ sigmoid activations to the output neuron.

layers.Dense(units=1, activation='linear')
layers.Activation('sigmoid')



For more details on CNNs: https://cs231n.github.io/convolutional-networks/
Dataset overview




In [ ]:
from tensorflow import keras

# import modules that will make it easy to build and train the linear regression model


# let's setup a sequential model -- which means that the model will be feed 
# forward (all models in this course will be sequential)


# the input layer specifies the shape of the input data to the model. Since our input 
# features, x, are 1-dimensional, we can set this value to 1. All models require
# some input layer (where you feed in data).


# a dense connection to 1 unit means that the inputs are fully connected to the 
# output neuron (in this case a single output neuron). Linear is the default 
# activation, but here we explicitly write it.


After we build the model, we can print a summary that shows the order of the layers, the shapes and number of parameters:

In [ ]:
model.summary()

### Build loss function

For both training and evaluation, we need to define a loss function that measures how closely the model's predictions match the target classes. This is usually a log-likelihood function for supervised learning. The log-likelihood for a binary classification is the binary cross-entropy loss function:

$\mathcal{L} = \sum_i y_i \log(p_i) + (1-y_i)\log(1-p_i)$

Fortunately, Keras makes this super easy as you'll see below.

### Build optimizer

Now that we have our loss function established, we need to be able to calculate the gradients of the loss function with respect to the parameters of the model, which is efficiently calculated with backpropagation. Then, we can update the weights with gradients scaled by the learning rate, which is known as gradient descent learning. Fortunately, tensorflow/Keras provides a nice wrapper to perform all of this. There are many built in optimizers: sgd, momentum, adam, adagrad, rmsprop. ADAM is the best introductory optimizer for most tasks. It performs an adaptive learning rate based on previous gradients and hence it is much faster to converge and leads to consistently good solutions.  

For more info on adam: https://towardsdatascience.com/adam-latest-trends-in-deep-learning-optimization-6be9a291375c

In [ ]:
# Now that we built the model, we have to set up the loss function and the optimizer.
# Since it's a linear regression, we will set it to 'mean_squared_error'. 
# we will set the optimizer to stochastic gradient descent 'sgd' (even though this
# is unnecessary for a linear regression)



# a short cut is: model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auroc])

### Fit the model

We want to train our CNN model with mini-batch stochastic gradient descent. So, we need a way to generate mini-batches of data. Keras handles all of this for you through the setting batch_size. It also lets you select the number of epochs to train.  You can also monitor validation performance by adding the validation_data flag.  

In [ ]:
# Now let's fit the model. The main inputs are x, y, epochs, which sets the number 
# of times to run through the dataset with SGD. Also, we set verbose to false,
# because too many outputs are written and it is overwhelming for a linear model.
# We will turn this flag to true for other datasets later.


### Evaluate model

Evaluating the test performance is easy with Keras. See below:

In [ ]:
# evaluate model


Now while these saliency plots seem to find a motif pattern that resembles the CTCF motif (see: https://www.factorbook.org/tf/human/CTCF/motif/ENCSR000BIE), the saliency maps are really noisy.  From my experience, this arises because of overfitting, which leads to spurious saliency maps as well -- here overfitting does not affect classification performance on the test set and is termed benign overfitting.


# Redo analysis

Let's redo this analysis with some regularization added to our CNN model. A very effective regularization is dropout and early stopping. For convolutional layers, let's add a dropout rate of 0.2 and for dense layers let's use a dropout of 0.5.  For early stopping, there is a built in "callback" in keras. Callbacks are objects taht perform actions during training. The early stopping call back checks whether the model's validation performance hasn't improved for x number of epochs. if the x number of epochs exceeds your patience, then training loop is stopped.  See below for example code.

In [ ]:
from tensorflow import keras
from tensorflow.keras import models, layers

# build model


# compile model


# create a callback for early stopping. Now we can increase the number of epochs 
# as early stopping will automatically stop training earlier


# fit model


Let's evaluate the model

In [ ]:
# evaluate model


# Validating the model

To validate what the network is learning let's calculate a saliency map --  derivative of the output neuron with respect to the inputs. Luckily, tensorflow makes this easy because it calculates gradients using automatic differentiation. We can calculate the gradient with the function model.optimizer.get_gradients. We need to create a keras function to execute this. See code below:


In [ ]:
import tensorflow
# create an op to calculate gradients of outputs with respect to inputs (i.e. saliency map)
# Note that we are taking the derivative of the pre-activated output layer (not the output layer)
# this is because the sigmoid can make the gradients very small when predictions are close to 1 or 0



Now we should choose which sequences we want to perform saliency analysis on. Let's choose the top 10 test sequences with the highest predictions. To find these, we can first predict the test sequences, sort them in descending order, then slice the top ten.

In [ ]:
# To get model predictions, we can simply call predict.


# sort predictions in decending order


# get the top num_plot predictions


# calculate saliency maps


Let's visualize the saliency maps. We can plot this as a heat map:

This isn't a very nice visualization. Fortunately for you, [Logomaker](https://logomaker.readthedocs.io/en/latest/) (by Tareen and Kinney) can help us visualize saliency maps. We need to first install it on Google Colab.

In [ ]:
!pip install logomaker

I have created a function to visualize the saliency maps.  You are welcome to take a look at it. It essentially just converts the saliency map in the format needed by logomaker. 

In [ ]:
import pandas as pd
import logomaker 

def plot_saliency_map(scores, alphabet, ax=None):
  L,A = scores.shape
  counts_df = pd.DataFrame(data=0.0, columns=list(alphabet), index=list(range(L)))
  for a in range(A):
    for l in range(L):
      counts_df.iloc[l,a] = scores[l,a]

  if not ax:
    ax = plt.subplot(1,1,1)
  logomaker.Logo(counts_df, ax=ax)


saliency_scores = (saliency_map * X).numpy()  # grad * input
for scores in saliency_scores:
  fig = plt.figure(figsize=(10,1))
  ax = plt.subplot(1,1,1)
  plot_saliency_map(scores, alphabet, ax)